In [12]:
import pandas as pd
import sqlite3
import os

check_list_database = r'C:\Users\hofong\Desktop\python\Python working station\Database\TDC_Check_List_database.db'
TDC_BR_Data_database = r'C:\Users\hofong\Desktop\python\Python working station\Database\TDC_BR_Data_database.db'
folder = r'C:\Users\hofong\Desktop\python\Python working station\Import files to DB'
csv_container = r'C:\Users\hofong\Desktop\python\Python working station\Import files to DB\Event Details Extraction'

if not os.path.exists(csv_container):
    os.makedirs(csv_container)

conn = sqlite3.connect(check_list_database)
cursor = conn.cursor()

file_list=[]
file_names=[]

excel_dic = {}



for root, directory, files in os.walk(folder):
    for file in files:
        if file.endswith('.xlsx'):
            file_path = os.path.join(root, file)

            

            #check if targeted worksheet is in the excel file, only extract data that is relevant if yes.
            try:
                xls = pd.ExcelFile(file_path)
                if 'Event Details' in xls.sheet_names:
                    df = pd.read_excel(file_path, sheet_name='Event Details', skiprows=range(4))
                    headers = df.columns.tolist()
                    revised_header = [header.replace(' ', '_').replace('(', '').replace(')', '').replace(',', '_').replace('?', '') for header in headers] 
                    df.columns = revised_header
                    change_to_csv_format = (os.path.join(csv_container, file.replace('.xlsx', '.csv') ))
                    df.to_csv(change_to_csv_format, index=False, encoding='utf_8_sig')
                    file_list.pop(file_list.index(file_path))

                elif 'Doc-1-Summary' in xls.sheet_names and 'High Risk Non Local' in xls.sheet_names:
                    df1 = pd.read_excel(file_path, sheet_name='Doc-1-Summary', skiprows=range(9))
                    headers1 = df1.columns.tolist()
                    revised_header1 = [header1.replace(' ', '_').replace('(', '').replace(')', '').replace(',', '_').replace('?', '') for header1 in headers1] 
                    df1.columns = revised_header1
                    change_to_csv_format1 = (os.path.join(csv_container, file.replace('.xlsx', '_HK.csv') ))
                    df1.to_csv(change_to_csv_format1, index=False, encoding='utf_8_sig')

                    df2 = pd.read_excel(file_path, sheet_name='High Risk Non Local')
                    headers2 = df2.columns.tolist()
                    revised_header2 = [header2.replace(' ', '_').replace('(', '').replace(')', '').replace(',', '_').replace('?', '') for header2 in headers2] 
                    df2.columns = revised_header2
                    change_to_csv_format1 = (os.path.join(csv_container, file.replace('.xlsx', '_OS.csv') ))
                    df2.to_csv(change_to_csv_format1, index=False, encoding='utf_8_sig')
                    file_list.pop(file_list.index(file_path))
                
                elif 'High Risk Non Local' in xls.sheet_names:
                    df = pd.read_excel(file_path, sheet_name='High Risk Non Local')
                    headers = df.columns.tolist()
                    revised_header = [header.replace(' ', '_').replace('(', '').replace(')', '').replace(',', '_').replace('?', '') for header in headers] 
                    df.columns = revised_header
                    change_to_csv_format = (os.path.join(csv_container, file.replace('.xlsx', '_OS.csv') ))
                    df.to_csv(change_to_csv_format, index=False, encoding='utf_8_sig')
                    file_list.pop(file_list.index(file_path))

                else: 
                    df = pd.read_excel(file_path)
                    headers = df.columns.tolist()
                    revised_header = [header.replace(' ', '_').replace('(', '').replace(')', '').replace(',', '_').replace('?', '') for header in headers] 
                    df.columns = revised_header
                    change_to_csv_format = (os.path.join(csv_container, file.replace('.xlsx', '.csv') ))
                    df.to_csv(change_to_csv_format, index=False, encoding='utf_8_sig')


            except:
                pass

        if file.endswith('.csv'):
            file_path = os.path.join(root, file)
            file_list.append(file_path)
            file_name = file.replace(" ","_")
            file_names.append(file_name)
            excel_dic = {file_name:file_list for file_name, file_list in zip(file_names, file_list)}

for file_name, excel_path in excel_dic.items():
    df = pd.read_csv(excel_path)
    sql_revised_header = df.columns
    preparing_header1 = '`'+'` TEXT, `'.join(sql_revised_header)+'` TEXT'
    preparing_header2 = preparing_header1.split(',')
    preparing_header3 =[]
    
    #change data type
    for header in preparing_header2:
        if 'No.' in header or '$' in header or 'Total' in header or 'Booth_Area' in header or 'Net' in header or 'Charges' in header or 'Calculated' in header or 'Branch_Number' in header or 'Net_Order' in header or 'C&E' in header or 'SME' in header or 'product_listing' in header or 'Extended_online' in header or 'booth' in header or 'Package' in header or 'Price' in header or 'Calculated' in header or 'Amount' in header or 'charges' in header or 'eFair_Catalogue' in header or 'Price' in header or 'Unnamed' in header or 'Diff' in header:
            header = header.replace('TEXT', 'INTEGER')
            preparing_header3.append(header)

        else:
            preparing_header3.append(header)
    cursor.execute (f'CREATE TABLE IF NOT EXISTS `{file_name}` ( {' ,'.join(preparing_header3)});')
    df.to_sql(file_name, conn, if_exists='append', index=False)
    print(f"Successfully imported {file_name} into Database!")

for file_name, excel_path in excel_dic.items():
    cursor.execute (f'ALTER TABLE `{file_name}` RENAME TO `HKTDC Fair {file_name}`')

        



conn.close

c:\Users\hofong\Desktop\python\Python working station\.venv\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)
c:\Users\hofong\Desktop\python\Python working station\.venv\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


Successfully imported HKTDC_HK_Baby_Products_Fair_2023_-_Govt_subsidy_w_16_Mar_2023_(with_address)_HK.csv into Database!
Successfully imported HKTDC_HK_Baby_Products_Fair_2023_-_Govt_subsidy_w_16_Mar_2023_(with_address)_OS.csv into Database!
Successfully imported Hong_Kong_International_Stationery_&_School_Supplies_Fair_2023_-_Govt_subsidy_w_(added_non_local_exhibitor_202303010)_Checked_and_no_abuse_case.csv into Database!
Successfully imported W&C_2023_-_checking_Exhibitor_List_with_currency_-_by_order_no.BR_expiry_(back_up).csv into Database!
Successfully imported Wine_&_Spirits_Fair_(Special_Edition)_-_Govt_subsidy_w_20_Jan_2023_(with_address)_no_abuse.csv into Database!


<function Connection.close()>